Total Game Score Model - Hyperparameter Tuning - GBM

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split

pd.options.display.max_rows = 10
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Load Data

ModellingDataLoader class

In [2]:
# from modelling_data_preparation import DataLoader

In [3]:
data = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/total-points-score-model/data/modelling-data/modelling_data_total_team_score.csv")

In [4]:
data.tail(2)

,Unnamed: 0,Match_ID,AFL_API_Match_ID,Home_Team,Away_Team,Date,Venue,Match_Status,Q4_Score,Round_ID,Year,Q1_Score,Q2_Score,Q3_Score,Margin,Total_Game_Score,Home_Win,City,Attendance,Umpires,Home_Coach_ID,Away_Coach_ID,Q5_Score,Temperature,Weather_Type,Home_Score,Away_Score,Home_Goals,Home_Behinds,Home_Total_Score,Away_Goals,Away_Behinds,Away_Total_Score,random5,training_set,Home_Total_Game_Score_avg2,Home_For_Goals_avg2,Home_For_Behinds_avg2,Home_For_Scores_avg2,Home_For_Total_Score_avg2,Home_Against_Goals_avg2,Home_Against_Behinds_avg2,Home_Against_Scores_avg2,Home_Against_Total_Score_avg2,Home_Total_Game_Score_avg10,Home_For_Goals_avg10,Home_For_Behinds_avg10,Home_For_Scores_avg10,Home_For_Total_Score_avg10,Home_Against_Goals_avg10,Home_Against_Behinds_avg10,Home_Against_Scores_avg10,Home_Against_Total_Score_avg10,Away_Total_Game_Score_avg2,Away_For_Goals_avg2,Away_For_Behinds_avg2,Away_For_Scores_avg2,Away_For_Total_Score_avg2,Away_Against_Goals_avg2,Away_Against_Behinds_avg2,Away_Against_Scores_avg2,Away_Against_Total_Score_avg2,Away_Total_Game_Score_avg10,Away_For_Goals_avg10,Away_For_Behinds_avg10,Away_For_Scores_avg10,Away_For_Total_Score_avg10,Away_Against_Goals_avg10,Away_Against_Behinds_avg10,Away_Against_Scores_avg10,Away_Against_Total_Score_avg10
3004,3005,2022F3_Sydney_Collingwood,CD_M20220142602,Sydney,Collingwood,2022-09-17 16:45:00,S.C.G.,CONCLUDED,14.11.95 - 14.10.94,2022F3,2022,6.3.39 - 3.0.18,11.7.73 - 7.1.43,13.10.88 - 10.5.65,1,189,1.0,Sydney,45608.0,NaN,John_Longmire,Craig_McRae,NaN,23.0,MOSTLY_SUNNY,14.11.95,14.10.94,14,11,95,14,10,94,5,False,174.5,14.0,9.0,23.0,93.0,12.0,9.5,21.5,81.5,172.1,14.4,13.1,27.5,99.5,10.5,9.6,20.1,72.6,163.5,12.5,11.5,24.0,86.5,11.5,8.0,19.5,77.0,161.0,12.0,9.3,21.3,81.3,11.6,10.1,21.7,79.7
3005,3006,2022F4_Geelong_Sydney,CD_M20220142701,Geelong,Sydney,2022-09-24 14:30:00,M.C.G.,CONCLUDED,20.13.133 - 8.4.52,2022F4,2022,6.5.41 - 1.0.6,9.8.62 - 4.2.26,15.11.101 - 4.3.27,81,185,1.0,Melbourne,100024.0,"Simon Meredith (439), Matt Stevic (449), Brend...",Chris_Scott,John_Longmire,NaN,15.0,RAIN,20.13.133,8.4.52,20,13,133,8,4,52,4,True,177.0,19.0,12.5,31.5,126.5,7.5,5.5,13.0,50.5,168.8,15.7,12.5,28.2,106.7,9.1,7.5,16.6,62.1,187.0,11.0,7.5,18.5,73.5,17.0,11.5,28.5,113.5,172.5,14.0,12.1,26.1,96.1,11.0,10.4,21.4,76.4


Clean Data

In [5]:
def preprocess_modelling_data(data):
    
    modelling_data = data[data['Date'] < "2019-01-01"]
    oot_data = data[data['Date'] >= "2019-01-01"]
    
    training_data = modelling_data[modelling_data['training_set'] == True]
    test_data = modelling_data[modelling_data['training_set'] == False]
    
    return training_data, test_data, oot_data

In [6]:
training_data, test_data, oot_data = preprocess_modelling_data(data)

Preprocessor class

In [7]:
# from preprocessor.py import Preprocessor

Train Model

Store these constants in a dataclass - ModellingConstants

In [8]:
response = "Total_Game_Score"

In [9]:
list(training_data)

['Unnamed: 0',
 'Match_ID',
 'AFL_API_Match_ID',
 'Home_Team',
 'Away_Team',
 'Date',
 'Venue',
 'Match_Status',
 'Q4_Score',
 'Round_ID',
 'Year',
 'Q1_Score',
 'Q2_Score',
 'Q3_Score',
 'Margin',
 'Total_Game_Score',
 'Home_Win',
 'City',
 'Attendance',
 'Umpires',
 'Home_Coach_ID',
 'Away_Coach_ID',
 'Q5_Score',
 'Temperature',
 'Weather_Type',
 'Home_Score',
 'Away_Score',
 'Home_Goals',
 'Home_Behinds',
 'Home_Total_Score',
 'Away_Goals',
 'Away_Behinds',
 'Away_Total_Score',
 'random5',
 'training_set',
 'Home_Total_Game_Score_avg2',
 'Home_For_Goals_avg2',
 'Home_For_Behinds_avg2',
 'Home_For_Scores_avg2',
 'Home_For_Total_Score_avg2',
 'Home_Against_Goals_avg2',
 'Home_Against_Behinds_avg2',
 'Home_Against_Scores_avg2',
 'Home_Against_Total_Score_avg2',
 'Home_Total_Game_Score_avg10',
 'Home_For_Goals_avg10',
 'Home_For_Behinds_avg10',
 'Home_For_Scores_avg10',
 'Home_For_Total_Score_avg10',
 'Home_Against_Goals_avg10',
 'Home_Against_Behinds_avg10',
 'Home_Against_Scores_avg10

In [10]:
modelling_features = [
    'Venue',
    'Year',
    'Temperature',
    'Weather_Type',
    'Home_Team',
    'Away_Team',
    'Home_Total_Game_Score_avg2',
    'Home_For_Goals_avg2',
    'Home_For_Behinds_avg2',
    'Home_For_Scores_avg2',
    'Home_For_Total_Score_avg2',
    'Home_Against_Goals_avg2',
    'Home_Against_Behinds_avg2',
    'Home_Against_Scores_avg2',
    'Home_Against_Total_Score_avg2',
    'Home_Total_Game_Score_avg10',
    'Home_For_Goals_avg10',
    'Home_For_Behinds_avg10',
    'Home_For_Scores_avg10',
    'Home_For_Total_Score_avg10',
    'Home_Against_Goals_avg10',
    'Home_Against_Behinds_avg10',
    'Home_Against_Scores_avg10',
    'Home_Against_Total_Score_avg10',
    'Away_Total_Game_Score_avg2',
    'Away_For_Goals_avg2',
    'Away_For_Behinds_avg2',
    'Away_For_Scores_avg2',
    'Away_For_Total_Score_avg2',
    'Away_Against_Goals_avg2',
    'Away_Against_Behinds_avg2',
    'Away_Against_Scores_avg2',
    'Away_Against_Total_Score_avg2',
    'Away_Total_Game_Score_avg10',
    'Away_For_Goals_avg10',
    'Away_For_Behinds_avg10',
    'Away_For_Scores_avg10',
    'Away_For_Total_Score_avg10',
    'Away_Against_Goals_avg10',
    'Away_Against_Behinds_avg10',
    'Away_Against_Scores_avg10',
    'Away_Against_Total_Score_avg10']

In [11]:
def get_modelling_features(data, feature_list, response):
    
    X, y = data[feature_list], data[response]
    
    return X, y

In [12]:
X_train, y_train = get_modelling_features(training_data, modelling_features, response)
X_test, y_test = get_modelling_features(test_data, modelling_features, response)
X_oot, y_oot = get_modelling_features(oot_data, modelling_features, response)

In [13]:
def one_hot_encode_fit(X):
    
    X_dummies = pd.get_dummies(X)
    
    return list(X_dummies)

In [14]:
def one_hot_encode_transform(X, expected_features):
    
    X_dummies = pd.get_dummies(X)
    
    for col in list(expected_features):
        if col not in list(X_dummies):
            X_dummies[col] = 0
            
    X_dummies = X_dummies[expected_features]
    
    return X_dummies

In [15]:
ohe_expected_features = one_hot_encode_fit(X_train)
X_train_dummies = one_hot_encode_transform(X_train, expected_features=ohe_expected_features)
X_test_dummies = one_hot_encode_transform(X_test, expected_features=ohe_expected_features)
X_oot_dummies = one_hot_encode_transform(X_oot, expected_features=ohe_expected_features)

In [16]:
X_train_dummies.shape, X_test_dummies.shape, X_oot_dummies.shape, 

((1909, 104), (476, 104), (621, 104))

Feature Selection

In [17]:
# from feature_selector import XGBFeatureSelector

In [18]:
def feature_selector(X_dummies, y, min_features = 20):
    
    import xgboost as xgb
    from sklearn.feature_selection import RFECV
    
    xgb_model = xgb.XGBRegressor()
    
    selector = RFECV(xgb_model, step=1, cv=5, min_features_to_select = min_features)
    
    selector = selector.fit(X_dummies, y)
        
    return list(selector.get_feature_names_out())

In [19]:
selected_feature_list = feature_selector(X_train_dummies, y_train)

In [20]:
selected_feature_list

['Home_Total_Game_Score_avg2',
 'Home_Against_Total_Score_avg2',
 'Home_For_Goals_avg10',
 'Home_For_Behinds_avg10',
 'Home_For_Total_Score_avg10',
 'Home_Against_Goals_avg10',
 'Home_Against_Behinds_avg10',
 'Home_Against_Scores_avg10',
 'Home_Against_Total_Score_avg10',
 'Away_Total_Game_Score_avg2',
 'Away_For_Goals_avg2',
 'Away_For_Scores_avg2',
 'Away_For_Total_Score_avg2',
 'Away_Against_Behinds_avg2',
 'Away_Against_Scores_avg2',
 'Away_Against_Total_Score_avg2',
 'Away_Total_Game_Score_avg10',
 'Away_For_Goals_avg10',
 'Away_For_Behinds_avg10',
 'Away_For_Scores_avg10',
 'Away_Against_Goals_avg10',
 'Away_Against_Behinds_avg10',
 'Away_Against_Scores_avg10',
 'Away_Against_Total_Score_avg10',
 'Venue_Docklands',
 'Venue_Marrara Oval',
 'Venue_Perth Stadium',
 'Weather_Type_CLEAR_NIGHT',
 'Weather_Type_MOSTLY_CLEAR',
 'Weather_Type_RAIN',
 'Home_Team_Essendon',
 'Away_Team_Adelaide',
 'Away_Team_Collingwood',
 'Away_Team_Sydney',
 'Away_Team_West Coast',
 'Away_Team_Western Bul

In [21]:
def get_selected_feature_data(X_dummies, selected_features):
    
    X_selected = X_dummies[selected_features]
    
    return X_selected

In [22]:
X_train_selected = get_selected_feature_data(X_train_dummies, selected_feature_list)
X_test_selected = get_selected_feature_data(X_test_dummies, selected_feature_list)
X_oot_selected = get_selected_feature_data(X_oot_dummies, selected_feature_list)

In [23]:
def get_dmatrix(X_selected, y):
    
    return xgb.DMatrix(X_selected, label=y)

In [24]:
dtrain = get_dmatrix(X_train_selected, y_train)
dtest = get_dmatrix(X_test_selected, y_test)
doot = get_dmatrix(X_oot_selected, y_oot)

Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [47]:
from hyperparameter_tuning import XGBHyperparameterTuner

In [48]:
xgb_tuner = XGBHyperparameterTuner(X_train_selected, y_train)

In [49]:
# xgb_tuner.trials

In [50]:
xgb_tuner.tune_hyperparameters()

[I 2023-05-08 13:46:10,752] A new study created in memory with name: no-name-e2d11880-ec78-4de5-bede-fd53ed82d3a0
[I 2023-05-08 13:46:10,838] Trial 0 finished with value: 182.10042980436123 and parameters: {'max_depth': 10, 'min_child_weight': 2, 'eta': 0.0005239106241761477, 'gamma': 20.0, 'lambda': 0.001285083154535903, 'alpha': 0.22986932622086753, 'subsample': 0.5426023454918614, 'colsample_bytree': 0.24207474381412536}. Best is trial 0 with value: 182.10042980436123.
/Users/ciaran/opt/anaconda3/envs/afl-total-model-env/lib/python3.9/site-packages/optuna/distributions.py:716: UserWarning: The distribution is specified by [2, 20] and step=5, but the range is not divisible by `step`. It will be replaced by [2, 17].
  warnings.warn(
[I 2023-05-08 13:46:10,849] Trial 1 finished with value: 167.27900421810415 and parameters: {'max_depth': 14, 'min_child_weight': 7, 'eta': 0.00765783329113083, 'gamma': 20.0, 'lambda': 0.6987365620719516, 'alpha': 0.9757835683487616, 'subsample': 0.337019

Number of finished trials:  100
Best trial:
  Value: 17.45326308084275
  Params: 
    max_depth: 2
    min_child_weight: 12
    eta: 0.41294254112407514
    gamma: 20.0
    lambda: 0.35560233747051745
    alpha: 0.05358772178288147
    subsample: 0.27518999577189024
    colsample_bytree: 0.3430224483178065


In [51]:
xgb_tuner.get_best_params()

{'max_depth': 2,
 'min_child_weight': 12,
 'eta': 0.41294254112407514,
 'gamma': 20.0,
 'lambda': 0.35560233747051745,
 'alpha': 0.05358772178288147,
 'subsample': 0.27518999577189024,
 'colsample_bytree': 0.3430224483178065}

In [ ]:
def train_xgboost_model(xgb_params, dtrain, dtest, objective, eval_metric, num_round = 100, early_stopping_rounds = 10):
    
    xgb_params['objective'] = objective
    xgb_params['eval_metric'] = eval_metric

    evallist = [(dtrain, 'train'), (dtest, 'test')]
    
    xgb_bst = xgb.train(xgb_params, dtrain, num_round, evallist, early_stopping_rounds=early_stopping_rounds)
    
    return xgb_bst

In [ ]:
xgb_params = study.best_trial.params
objective = 'reg:squarederror'
eval_metric = 'rmse'

In [ ]:
xgb_bst = train_xgboost_model(xgb_params, dtrain, dtest, objective, eval_metric, num_round = 100, early_stopping_rounds = 10)

In [ ]:
xgb_bst

In [ ]:
xgb.plot_importance(xgb_bst, importance_type = "total_gain")

In [ ]:
xgb.plot_importance(xgb_bst, importance_type = "gain")

In [ ]:
xgb.plot_importance(xgb_bst, importance_type = "weight")

Evaluate Model

In [ ]:
def get_xgb_predictions(data, xgb_bst):
    
    X_dummies = one_hot_encode_transform(data, expected_features=xgb_bst.feature_names)
    
    dmatrix = xgb.DMatrix(X_dummies)
    
    xgb_preds = bst.predict(dmatrix)
    
    return xgb_preds

In [ ]:
test_preds = get_xgb_predictions(test_data, xgb_bst)
oot_preds = get_xgb_predictions(oot_data, xgb_bst)

In [ ]:
def MAE(data, model, actuals):

    preds = model.predict(data)
    absolute_error = abs(preds - actuals)
    mae = np.mean(absolute_error)

    return mae

In [ ]:
def RMSE(data, model, actuals):

    preds = model.predict(data)
    squared_error = (preds - actuals)**2
    mean_squared_error = np.mean(squared_error)
    rmse = (mean_squared_error)**0.5

    return rmse

In [ ]:
def get_error_metrics(data, model, actuals):

    mae = MAE(data, model, actuals)
    rmse = RMSE(data, model, actuals)

    print("MAE: {}".format(mae))
    print("RMSE: {}".format(rmse))

    return mae, rmse

In [ ]:
get_error_metrics(dtrain, xgb_bst, y_train)

In [ ]:
get_error_metrics(dtest, xgb_bst, y_test)

In [ ]:
get_error_metrics(doot, xgb_bst, y_oot)

Save model predictions

In [ ]:
data['xgb_preds'] = get_xgb_predictions(data, xgb_model)

In [ ]:
data['xgb_preds'].mean(), data[response].mean()

In [ ]:
data['std_dev'] = np.std(y_train)

In [ ]:
data[['Match_ID', 'Date', 'Total_Game_Score', 'xgb_preds', 'std_dev']].head()

In [ ]:
data[['Match_ID', 'Date', 'Total_Game_Score', 'xgb_preds', 'std_dev']].to_csv("/Users/ciaran/Documents/Projects/AFL/total-points-score-model/data/model-predictions/xgb_model_preds.csv", index=False)